In [1]:
# imports
import os
from solarprophet import TabularTest
import constants
import utils
import tensorflow as tf
import pandas as pd

In [4]:
# inputs
model_name = "SOL-210_CV1"

# preprocess date
tt = TabularTest(
    # Required Parameters
    n_steps_in=1,
    n_steps_out=12,
    selected_features=[
        "BRBG Total Cloud Cover [%]",
        "CSI GHI",
        "dni t-4",
        # "940nm POM-01 Photometer [uA]",
        "cs_dev_mean t-10 dni",
        # "675nm POM-01 Photometer [uA]",
        "CSI DNI",
        "elevation",
        "cs_dev t dni",
        "dni t-9",
    ],
    selected_groups=None,
    selected_responses=["Delta CSI GHI"],
    scaler_type="minmax",
    data_path=constants.JOINT_DATA_H5_PATH,
    ## Saving and cache parameters (optional)
    model_save_path="results",
    datetimes_cache_path=constants.DATETIME_VALUES_PATH,
    window_cache_path=os.path.join("data", "windows_cache.h5"),
    all_past_features=constants.PAST_FEATURES,
    all_future_features=constants.FUTURE_FEATURES,
    all_scalar_responses=constants.SCALAR_RESPONSES,
    all_relative_responses=constants.RELATIVE_RESPONSES,
    # Model (optional)
    model=None,
    model_name="TimeDistributed_CNN-LSTM-MLP",
    # Optimizer and training tuning parameters (optional)
    scale_responses=True,
    epochs=3000,
    shuffle_training_order=False,
    batch_size=4000,
    loss="mae",
    optimizer=None,
    learning_rate=tf.optimizers.schedules.CosineDecayRestarts(
        1e-2,  # initial learning rate
        500,  # number of steps to initially decay over
        t_mul=4,  # number of times to restart
        m_mul=0.5,  # restart maximum learning rate multiplier
        alpha=0,
    ),
    callbacks=[],
    early_stopping=True,
    stopping_patience=1000,
    stopping_min_delta=1e-4,
    metrics=[],  # todo possibly replace
    fit_verbose=1,
    # Utility Parameters (optional)
    data_cols=constants.DATA_COLS,
    feature_groups=constants.FEATURE_GROUPS,
    scalar_response=constants.SCALAR_RESPONSES,
    relative_response=constants.RELATIVE_RESPONSES,
    seed=42,
    n_job_workers=1,
    # Neptune Parameters (optional)
    neptune_log=False,
    neptune_run_name=f"Fine Tuning Feature number",
    tags=["CV2", "Time Steps"],
)
original_epochs = tt.epochs

train_validate_dates = ["2020-09-27"]
end_dates = ["2021-09-27"]
for i in range(len(train_validate_dates)):
    print(f" Starting CV 2 ".center(40, "="))
    tt.import_preprocess_cached_windows(
        train_validate_date=train_validate_dates[i], end_date=end_dates[i], verbose=2
    )

    # compile the model to fit for 500 epochs with mse then mae

    remaining_epochs = max(500, original_epochs - 500)
    tt.loss = "mse"
    tt.epochs = 500
    tt.compile_model()
    tt.fit_model()

    # extract the weights and recompile with new loss function
    weights = tt.model.get_weights()
    tt.loss = "mae"
    tt.epochs = remaining_epochs
    tt.compile_model()
    tt.fit_model()

    print(" Fit Complete ".center(40, "-"))
    print(" Calculating Error Metrics ".center(40, "-"))
    tt.final_error_metrics(cv=i + 1)
    print(" Error Metrics Complete ".center(40, "-"))
    print(" Saving ".center(40, "-"))
    tt.save_and_quit(cv=i + 1)
    print(" Saved ".center(40, "-"))
    print(" Complete! ".center(40, "="))
# tt.run.stop()

============ Starting CV 2 =============
====== Reading windows from cache ======
past_features.shape: (78930, 1, 134)

Done
 Selecting and scaling relevant features and responses 
datetimes.shape: (78930, 13)
past_features.shape: (78930, 1, 134)
future_features.shape: (78930, 12, 7)
scalar_responses.shape: (78930, 12, 3)
relative_responses.shape: (78930, 12, 2)
clear_sky_indexes.shape: (78930, 13)
clear_sky_irradiances.shape: (78930, 13)
# selected past features: 8
# selected future features: 0
# selected scalar responses: 0
# selected relative responses: 1
selected_past_features_indices=[3, 17, 96, 114, 18, 71, 15, 111]
selected_future_features_indices=[]
selected_scalar_responses_indices=[]
selected_relative_responses_indices=[1]
train_dates.shape: (47733, 13)
train_past_features.shape: (47733, 1, 8)
train_future_features.shape: (47733, 12, 0)
train_scalar_responses.shape: (47733, 12, 0)
train_relative_responses.shape: (47733, 12, 1)
train_clear_sky_indexes.shape: (47733, 13)
train_

INFO:tensorflow:Assets written to: results\2023-12-14 18.46.45_CV1\assets


INFO:tensorflow:Assets written to: results\2023-12-14 18.46.45_CV1\assets


---------------- Saved -----------------
============== Complete! ===============


AttributeError: 'dict' object has no attribute 'stop'

In [5]:
tt.run

{'name': 'Fine Tuning Feature number',
 'selected features': ['BRBG Total Cloud Cover [%]',
  'CSI GHI',
  'dni t-4',
  'cs_dev_mean t-10 dni',
  'CSI DNI',
  'elevation',
  'cs_dev t dni',
  'dni t-9'],
 'selected groups': None,
 'selected responses': ['Delta CSI GHI'],
 'scaler type': 'minmax',
 'loss function': 'mae',
 'history': <keras.callbacks.History at 0x227839425b0>,
 'CV 1: Train MBE': 9.13998048772407,
 'CV 1: Validate MBE': 6.188000362706828,
 'CV 1: Train MBE t+10 min': 3.41123415027141,
 'CV 1: Validate MBE t+10 min': 1.1284263380335524,
 'CV 1: Train MBE t+20 min': 1.7712822594028996,
 'CV 1: Validate MBE t+20 min': -1.176042051425341,
 'CV 1: Train MBE t+30 min': 2.292337620296611,
 'CV 1: Validate MBE t+30 min': -0.7248699911718753,
 'CV 1: Train MBE t+40 min': 2.9764509114481017,
 'CV 1: Validate MBE t+40 min': -0.23041289245634083,
 'CV 1: Train MBE t+50 min': 6.155166566801758,
 'CV 1: Validate MBE t+50 min': 2.7794757681424977,
 'CV 1: Train MBE t+60 min': 7.104094